# DEPA Final Project Data Transformation/Preparation for Database
## William DeForest

In [230]:
# Import packages

import requests
import pandas as pd
import numpy as np
import csv

In [231]:
# Import data

teams = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\nba_teams.csv")
players = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\nba_players.csv")
player_info = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\player_info_combined.csv")
player_stats = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\career_stats.csv")
player_salaries = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\combined_salaries.csv", encoding="ISO-8859-1")
team_revenue = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\team_revenue.csv")
team_operating_income = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\team_operating_income.csv")
team_ticket_revenue = pd.read_csv(r"C:\Users\wdd72\OneDrive\Documents\UChicago\Fall '23\Engineering Platforms\Final Project\team_ticket_revenue.csv")

In [232]:
# Change team_revenue, team_operating_income and team_ticket_revenue to narrow format

# Melt the DataFrame to convert from wide to narrow format
team_revenue_narrow = pd.melt(team_revenue, id_vars=['TEAM_NAME'], var_name='SEASON', value_name='TEAM_REVENUE')


# Melt the DataFrame to convert from wide to narrow format
team_operating_income_narrow = pd.melt(team_operating_income, id_vars=['TEAM_NAME'], var_name='SEASON', value_name='TEAM_OPERATING_INCOME')


# Melt the DataFrame to convert from wide to narrow format
team_ticket_revenue_narrow = pd.melt(team_ticket_revenue, id_vars=['TEAM_NAME'], var_name='SEASON', value_name='TEAM_TICKET_REVENUE')


In [233]:
# Merge team financials tables

team_revenues = pd.merge(team_revenue_narrow, team_ticket_revenue_narrow, on=['TEAM_NAME', 'SEASON'])

team_financials = pd.merge(team_revenues, team_operating_income_narrow, on=['TEAM_NAME', 'SEASON'])

# Add a FINANCIALS_ID column

team_financials['FINANCIAL_ID'] = range(1, len(team_financials) + 1)

# Add the TEAM_ID column from teams table

team_financials = pd.merge(team_financials, teams[['id', 'full_name']], left_on=['TEAM_NAME'], right_on=['full_name'])
team_financials = team_financials.drop(['full_name'], axis=1)
team_financials = team_financials.drop(['TEAM_NAME'], axis=1)
team_financials = team_financials.rename(columns={'id': 'TEAM_ID'})
print(team_financials)

    SEASON  TEAM_REVENUE  TEAM_TICKET_REVENUE  TEAM_OPERATING_INCOME  \
0     2011           109                 26.3                    -15   
1     2012            99                 20.1                    -19   
2     2013           119                 23.6                     -4   
3     2014           133                 22.9                     15   
4     2015           142                 27.8                      7   
..     ...           ...                  ...                    ...   
295   2016           155                 34.8                      7   
296   2017           222                 55.1                     21   
297   2018           255                 56.8                     34   
298   2019           269                 60.9                     55   
299   2020           231                 42.6                     39   

     FINANCIAL_ID     TEAM_ID  
0               1  1610612737  
1              31  1610612737  
2              61  1610612737  
3      

In [234]:
# Join PLAYER_ID column from player_info table to player_salaries table by name=display_First_last

player_salaries = pd.merge(player_salaries, player_info[['PERSON_ID', 'DISPLAY_FIRST_LAST']], left_on=['Name'], right_on=['DISPLAY_FIRST_LAST'])

# Lose 1,063 salary obs. (out of 15,747) on merge. Most likely due to different name spellings.

In [235]:
player_salaries = player_salaries.rename(columns={'PLAYER_SEASON_SALARY_ID': 'SALARY_ID'})
player_salaries = player_salaries.rename(columns={'PERSON_ID': 'PLAYER_ID'})
player_salaries = player_salaries.drop(['DISPLAY_FIRST_LAST'], axis=1)

In [236]:
# Change the format of SEASON_ID column in player_stats to have just the first four digits

# Extract the first four digits from the 'SEASON_ID' column
player_stats['SEASON'] = player_stats['SEASON_ID'].str.slice(0, 4)
player_stats = player_stats.drop(['SEASON_ID'], axis=1)
player_stats['SEASON'] = player_stats['SEASON'].astype(int)

In [237]:
# Rename PLAYER_SEASON_ID to STATS_ID

player_stats = player_stats.rename(columns={'PLAYER_SEASON_ID': 'STATS_ID'})

In [238]:
# Rename PERSON_ID to PLAYER_ID and drop unnecessary columns

player_info = player_info.rename(columns={'PERSON_ID': 'PLAYER_ID'})
player_info = player_info.drop(['PLAYER_SLUG'], axis=1)
player_info = player_info.drop(['DISPLAY_FI_LAST'], axis=1)
player_info = player_info.drop(['DISPLAY_LAST_COMMA_FIRST'], axis=1)
player_info = player_info.drop(['FIRST_NAME'], axis=1)
player_info = player_info.drop(['LAST_NAME'], axis=1)
player_info = player_info.drop(['DISPLAY_FIRST_LAST'], axis=1)

# Create a DEMOGRAPHIC_ID column to be a PK
player_info['DEMOGRAPHIC_ID'] = range(1, len(player_info) + 1)

In [239]:
# Rename players table columns

players = players.rename(columns={'id': 'PLAYER_ID'})
players = players.rename(columns={'full_name': 'FULL_NAME'})
players = players.rename(columns={'first_name': 'FIRST_NAME'})
players = players.rename(columns={'last_name': 'LAST_NAME'})
players = players.rename(columns={'is_active': 'IS_ACTIVE'})

In [240]:
# Join TEAM_ID column from player_info table to players table

players = pd.merge(players, player_info[['PLAYER_ID', 'TEAM_ID']], on=['PLAYER_ID'], how='left')

In [241]:
# Rename teams table columns

teams = teams.rename(columns={'id': 'TEAM_ID'})
teams = teams.rename(columns={'full_name': 'TEAM_NAME'})
teams = teams.rename(columns={'abbreviation': 'ABR'})
teams = teams.rename(columns={'nickname': 'NICKNAME'})
teams = teams.rename(columns={'city': 'CITY'})
teams = teams.rename(columns={'state': 'STATE'})
teams = teams.rename(columns={'year_founded': 'YEAR_FOUNDED'})

## Export prepared datasets

In [242]:
# Export team to csv

teams.to_csv('team.csv', index=False)

In [243]:
# Export players to csv

players.to_csv('player.csv', index=False)

In [244]:
# Export player_demographics to csv

player_info.to_csv('demographic.csv', index=False)

In [248]:
# Export stats to csv

player_stats.to_csv('stats.csv', index=False)

In [246]:
# Export player_salaries to csv

player_salaries.to_csv('salary.csv', index=False)

In [247]:
# Export team_financials to csv

team_financials.to_csv('financial.csv', index=False)

In [249]:
player_info.dtypes

PLAYER_ID                             int64
BIRTHDATE                            object
SCHOOL                               object
COUNTRY                              object
LAST_AFFILIATION                     object
HEIGHT                               object
WEIGHT                              float64
SEASON_EXP                            int64
JERSEY                               object
POSITION                             object
ROSTERSTATUS                         object
GAMES_PLAYED_CURRENT_SEASON_FLAG     object
TEAM_ID                               int64
TEAM_NAME                            object
TEAM_ABBREVIATION                    object
TEAM_CODE                            object
TEAM_CITY                            object
PLAYERCODE                           object
FROM_YEAR                             int64
TO_YEAR                               int64
DLEAGUE_FLAG                         object
NBA_FLAG                             object
GAMES_PLAYED_FLAG               